In [1]:
from game import Game
import time
import os
import re
from heapq import heappop, heappush
from collections import deque
import time


## Load maps

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [3]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [4]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [5]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [6]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [7]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [8]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

In [10]:
STATE_LIMIT = int(8e5)

class state:
    def __init__ (self, player, boxes):
        self.player = player
        self.boxes = boxes
    def __eq__(self, other):
        return self.player == other.player and self.boxes == other.boxes

    def __hash__(self):
        return hash((self.player, frozenset(self.boxes)))

    def __lt__(self, other):
        return 0

### BFS

In [11]:
# TODO: Must return moves (if there is no solution return None), number of visited statesdef solver_bfs(game_map):
def solver_bfs(game_map):
    seen = set()

    game = Game(game_map)
    states = deque()
    moves =  deque()

    initSt = state(game.get_player_position(),game.get_box_locations())
    states.append(initSt)
    moves.append("")



    stateCount = 0
    while states:
        if stateCount >= STATE_LIMIT :
            return None, 0
        st = states.popleft()
        mv = moves.popleft()

        if st in seen:
            continue
        seen.add(st)
        stateCount += 1
        #game.display_map()
        #print("----------------------")
        game.set_box_positions(st.boxes)
        game.set_player_position(st.player)
        if game.is_game_won() :
            return mv, stateCount

        for dir in ['U','D','L','R']:

            if not  game.apply_move(dir):
                continue

            states.append(state(game.get_player_position(), game.get_box_locations()))
            moves.append(mv + dir)
            
            game.set_box_positions(st.boxes)
            game.set_player_position(st.player)

    return None, 0

### DFS

In [12]:
# TODO: Must return moves (if there is no solution return None), number of visited statesdef solver_bfs(game_map):
def solver_dfs(game_map, dlimit = -1):
    seen = set()

    game = Game(game_map)
    states = deque()
    moves =  deque()

    initSt = state(game.get_player_position(),game.get_box_locations())
    states.append((initSt,0))
    moves.append("")



    stateCount = 0
    while states:
        if stateCount >= STATE_LIMIT :
            return None, 0
        st,d = states.pop()
        mv = moves.pop()
        if st in seen:
            continue
        seen.add(st)
        stateCount += 1
        #game.display_map()
        #print("----------------------")
        game.set_box_positions(st.boxes)
        game.set_player_position(st.player)
        if game.is_game_won() :
            return mv, stateCount
        if d == dlimit :
            continue
        for dir in ['U','D','L','R']:

            if not  game.apply_move(dir):
                continue

            states.append((state(game.get_player_position(), game.get_box_locations()),d+1))
            moves.append(mv + dir)

            game.set_box_positions(st.boxes)
            game.set_player_position(st.player)

    return None, 0

### IDS

In [13]:
# TODO: Must return moves, number of visited states
MAX_DEPH = 200
TIME_LIMIT = 100

start = time.time()
def solver_ids(game_map):
    totalStates = 0
    for d in range(1,MAX_DEPH):
        if time.time() - start > TIME_LIMIT:
            break
        moves, cntStates = solver_dfs(game_map, d)
        totalStates += cntStates
        if moves != None:
            return moves , totalStates
    return None, 0

### A*

In [14]:
# TODO

def dis(a, b) :
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def heuristic(game):
    boxs = game.get_box_locations()
    goals = game.get_goal_locations()
    portals = game.get_portal_locations()

    totalDis = 0
    for i in range(len(boxs)):
        mini = dis(boxs[i], goals[i])
        for p1,p2 in portals:
            mini = min(mini, dis(p1, boxs[i]) + dis(p2, goals[i]))
            mini = min(mini, dis(p2, boxs[i]) + dis(p1, goals[i]))
        totalDis += mini

    return totalDis



    

# TODO: Must return moves, number of visited states
def solver_astar(game_map, heuristic_func=heuristic, weight=10):
    game = Game(game_map)
    initSt = state(game.get_player_position(),game.get_box_locations())

    minheap = []
    heappush(minheap, (0, initSt, ""))
    g = {initSt:0}
    stateCount = 0

    while minheap:
        if stateCount >= STATE_LIMIT :
            return None, 0
        _, st, mv = heappop(minheap)
        
        stateCount += 1
        game.set_box_positions(st.boxes)
        game.set_player_position(st.player)
        if game.is_game_won() :
            return mv, stateCount

        for dir in ['U','D','L','R']:
            if not  game.apply_move(dir):
                continue
            adj = state(game.get_player_position(),game.get_box_locations())
            if not adj in g or (g[adj]>g[st]+1):
                g[adj] = g[st]+1
                f = g[adj] + weight * heuristic_func(game)
                heappush(minheap, (f, adj, mv+dir))
            game.set_box_positions(st.boxes)
            game.set_player_position(st.player)
    
    return None, 0

## Solve

In [15]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
}

In [16]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [17]:
solve(9, "A*") # Solve map 1 using BFS

A* took 13.34 seconds on map 9 and visited 510181 states.
81 moves were used: RRLURRUUUULUULDLDRRURDDDDRRDDRDLDLURULDRRULDLLUURURDLDRDDLULURURRURDRDLDLLULURURD


In [18]:
def solve_all():
    for method in SOLVERS.keys():
        for map in range(min(map_file_indices), max(map_file_indices) + 1):
            solve(map, method)
            

In [19]:
solve_all() # Solve all maps using all methods

BFS took 0.0 seconds on map 1 and visited 44 states.
7 moves were used: UDDULRR
BFS took 0.0 seconds on map 2 and visited 21 states.
6 moves were used: LUDDUL
BFS took 0.0 seconds on map 3 and visited 122 states.
13 moves were used: ULDDUUUURDDDD
BFS took 0.0 seconds on map 4 and visited 0 states.
No Solution Found!
BFS took 0.09 seconds on map 5 and visited 6328 states.
15 moves were used: ULDDRDLLLUUURUL
BFS took 0.3 seconds on map 6 and visited 17807 states.
34 moves were used: UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR
BFS took 13.21 seconds on map 7 and visited 644754 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR
BFS took 0.09 seconds on map 8 and visited 7419 states.
14 moves were used: UURDLDRRDRURDR
BFS took 15.98 seconds on map 9 and visited 0 states.
No Solution Found!
BFS took 3.46 seconds on map 10 and visited 241503 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL
DFS took 0.0 seconds on map 1 and visited 11 states.
7 moves were used: RLL

## Report

*Question 1*

what is the defenition of state in this problem?

The position of the player and boxes are assumed to be the state of the problem.

if we take the whole plan as state it would take too much memory and time to process and compare states.







*Question 2*

how do you define the actions based on the states you defined?

actions are moving in one of the four directions up, down, left, right


*Question 3*

explain your way of modeling the problem?

initial state is where we are starting and basically the state the game is given to us
the goal state is where all boxes are on their coresponding positions.
the actions are moving in four directions of up, down, left, right which can move boxes if there are any in the path



*Question 4*

how can we improve the problem and search space?

some states can reach goal once they are reached. for example if a box is moved to a corner it can't be brought out of it.

so we can skip this states once we reach them.



*Question 5*

explain each of the implemented algorithms and their differences.

BFS always finds the shortest path to the answer because it expands nodes at each level
and that's exactly the reason that it takes too much memory

DFS tries to find a path by expanding the node at hand so the memory it takes is much less than BFS, yet it doesn't find the optimal solution and may take too much time to find one!

IDS has BFS optimality and DFS space minimization. it performs DFS to a certain level at each iteration

A* uses a heuristic function to estimate how close we are to a desired solution and it explands the "better" nodes


*Question 6*

DFS solutions are so longer than bfs
ids has optinmal solutions like bfs with less memory

*Question 7*

explain the heuristc you used in informed search and study them by the property of being admissible and consistent?

I used a combination of manhatan distance of the boxes and their goal and also the distance from the box to one side of the portal and distance from the other side of portal to the goal

then took the minimum of these two distances and calculated the sum over all boxes

it's consistent both consistent and admisable as it statisfies the triangle inequality and also there is no overestimation as the this heusitic reaches 0 when the box has reached the goal.




*Question 8*

comapre the algorithms 

bfs solutions are so clean and short,
for some maps the bfs is faster and for some dfs is faster

ids is faster than bfs and dfs in general 

a* is the fastest and finds fairly clean solutions 

*Quesion 9*

|       |  number of visited states | output | time |
|-------|----------|----------|----------|
|BFS	| 123000 | 21 | 3.8 |
|DFS	| 72000 | 2700 | 2 |
|IDS	| 113000 | 21 | 4 |
|A*  	| 1000 | 25 | 1.5 |